In [ ]:
import zipfile
import os

zip_path = '/content/drive/My Drive/shoulder.zip'  # 실제 파일 위치 맞게 경로 지정
extract_dir = '/content/drive/My Drive/shoulder_unzip2'  # 압축이 풀릴 폴더

# 압축 풀릴 폴더가 없으면 생성
os.makedirs(extract_dir, exist_ok=True)

# 압축 해제
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print('압축 해제 완료:', extract_dir)


압축 해제 완료: /content/drive/My Drive/shoulder_unzip2


In [ ]:
pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.12.0.88
    Uninstalling opencv

In [ ]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.18.0
3.8.0


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

img_path = '/content/drive/MyDrive/shoulder_unzip/정상/frame_00023.jpg'
img = cv2.imread(img_path)
img = cv2.resize(img, (480, 270))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

mp_pose = mp.solutions.pose

with mp_pose.Pose(static_image_mode=True, model_complexity=2) as pose:
    result = pose.process(img_rgb)
    if not result.pose_landmarks:
        print("[Error] Pose 랜드마크 인식 실패")
    else:
        lm = result.pose_landmarks.landmark
        h, w = img.shape[:2]

        # 모든 랜드마크 시각화
        annotated_img = img.copy()
        for i, landmark in enumerate(lm):
            x, y = int(landmark.x * w), int(landmark.y * h)
            cv2.circle(annotated_img, (x, y), 5, (255,0,0), -1)
            cv2.putText(annotated_img, str(i), (x, y-7), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,0), 1)
        save_path = '/content/frame_00023_allkeypoints.jpg'
        cv2.imwrite(save_path, annotated_img)
        print(f'모든 랜드마크 시각화 저장: {save_path}')


모든 랜드마크 시각화 저장: /content/frame_00023_allkeypoints.jpg


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

img_path = '/content/drive/MyDrive/shoulder_unzip/정상/frame_00023.jpg'
img = cv2.imread(img_path)
img = cv2.resize(img, (480, 270))
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

mp_pose = mp.solutions.pose

with mp_pose.Pose(static_image_mode=True, model_complexity=2) as pose:
    result = pose.process(img_rgb)
    if not result.pose_landmarks:
        print("[Error] Pose 랜드마크 인식 실패")
    else:
        lm = result.pose_landmarks.landmark
        h, w = img.shape[:2]

        # 왼팔
        l_shoulder = np.array([lm[11].x * w, lm[11].y * h])
        l_elbow    = np.array([lm[13].x * w, lm[13].y * h])
        l_wrist    = np.array([lm[15].x * w, lm[15].y * h])
        # 오른팔
        r_shoulder = np.array([lm[12].x * w, lm[12].y * h])
        r_elbow    = np.array([lm[14].x * w, lm[14].y * h])
        r_wrist    = np.array([lm[16].x * w, lm[16].y * h])

        annotated_img = img.copy()
        # 왼팔(빨강)
        for pt in [l_shoulder, l_elbow, l_wrist]:
            cv2.circle(annotated_img, (int(pt[0]), int(pt[1])), 7, (0,0,255), -1)
        cv2.line(annotated_img, (int(l_shoulder[0]), int(l_shoulder[1])), (int(l_elbow[0]), int(l_elbow[1])), (0,0,255), 3)
        cv2.line(annotated_img, (int(l_elbow[0]), int(l_elbow[1])), (int(l_wrist[0]), int(l_wrist[1])), (0,0,255), 3)
        # 오른팔(파랑)
        for pt in [r_shoulder, r_elbow, r_wrist]:
            cv2.circle(annotated_img, (int(pt[0]), int(pt[1])), 7, (255,0,0), -1)
        cv2.line(annotated_img, (int(r_shoulder[0]), int(r_shoulder[1])), (int(r_elbow[0]), int(r_elbow[1])), (255,0,0), 3)
        cv2.line(annotated_img, (int(r_elbow[0]), int(r_elbow[1])), (int(r_wrist[0]), int(r_wrist[1])), (255,0,0), 3)

        save_path = '/content/frame_00023_both_arms.jpg'
        cv2.imwrite(save_path, annotated_img)
        print(f'양팔 시각화 결과 저장: {save_path}')


양팔 시각화 결과 저장: /content/frame_00023_both_arms.jpg


In [ ]:
import cv2                          # OpenCV 라이브러리 임포트 (이미지 처리용)
import mediapipe as mp              # MediaPipe 라이브러리 임포트 (Pose 추정용)
import numpy as np                  # Numpy 라이브러리 임포트 (수치 연산용)
import os                           # OS 모듈 임포트 (파일/폴더 경로 관리용)
from glob import glob               # glob 함수 임포트 (파일 리스트 가져오기용)

base_dir = '/content/drive/MyDrive/shoulder_unzip'    # 원본 이미지가 들어있는 최상위 폴더 경로 (수정 필요)
output_base = '/content/shoulder_pose_vis'            # 시각화 결과 저장할 폴더 경로
categories = ['정상', '비정상']                        # 분류 카테고리(폴더명): 정상/비정상

mp_pose = mp.solutions.pose         # mediapipe pose 솔루션 객체

# 카테고리별(정상/비정상)로 반복
for category in categories:
    input_dir = os.path.join(base_dir, category)            # 입력 이미지 폴더 경로 설정
    output_dir = os.path.join(output_base, category)        # 결과 저장 폴더 경로 설정
    os.makedirs(output_dir, exist_ok=True)                  # 결과 폴더가 없으면 생성

    # jpg, png 확장자를 가진 이미지 파일 리스트 가져오기
    img_list = glob(os.path.join(input_dir, '*.jpg')) + glob(os.path.join(input_dir, '*.png'))

    # mediapipe pose 객체 생성 (정적 이미지 모드, 높은 복잡도)
    with mp_pose.Pose(static_image_mode=True, model_complexity=2) as pose:
        # 이미지 파일별로 반복
        for img_path in img_list:
            img = cv2.imread(img_path)                      # 이미지 읽기
            if img is None:                                 # 이미지가 안 읽힐 때
                print(f'[Error] 이미지 불러오기 실패: {img_path}')
                continue
            img = cv2.resize(img, (480, 270))               # 이미지 크기 480x270으로 resize
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # BGR->RGB 변환 (mediapipe는 RGB 필요)
            result = pose.process(img_rgb)                  # pose 추정 실행
            base_name = os.path.basename(img_path)          # 파일명만 추출
            save_path = os.path.join(output_dir, base_name) # 저장 경로 생성

            if not result.pose_landmarks:                   # 관절 인식 안 되면
                print(f'[Warning] Pose 인식 실패: {img_path}')
                cv2.imwrite(save_path, img)                 # 원본 이미지 저장
                continue

            lm = result.pose_landmarks.landmark             # 관절 좌표 가져오기
            h, w = img.shape[:2]                            # 이미지 높이, 너비

            # 왼팔 주요 관절 좌표 추출 (Shoulder, Elbow, Wrist) - mediapipe 번호 참고
            l_shoulder = np.array([lm[11].x * w, lm[11].y * h])
            l_elbow    = np.array([lm[13].x * w, lm[13].y * h])
            l_wrist    = np.array([lm[15].x * w, lm[15].y * h])
            # 오른팔 주요 관절 좌표 추출
            r_shoulder = np.array([lm[12].x * w, lm[12].y * h])
            r_elbow    = np.array([lm[14].x * w, lm[14].y * h])
            r_wrist    = np.array([lm[16].x * w, lm[16].y * h])

            annotated_img = img.copy()                      # 원본 이미지 복사 (시각화용)

            # 왼팔: 관절점(빨강) 찍고, 선(빨강) 그리기
            for pt in [l_shoulder, l_elbow, l_wrist]:
                cv2.circle(annotated_img, (int(pt[0]), int(pt[1])), 7, (0,0,255), -1)
            cv2.line(annotated_img, (int(l_shoulder[0]), int(l_shoulder[1])), (int(l_elbow[0]), int(l_elbow[1])), (0,0,255), 3)
            cv2.line(annotated_img, (int(l_elbow[0]), int(l_elbow[1])), (int(l_wrist[0]), int(l_wrist[1])), (0,0,255), 3)

            # 오른팔: 관절점(파랑) 찍고, 선(파랑) 그리기
            for pt in [r_shoulder, r_elbow, r_wrist]:
                cv2.circle(annotated_img, (int(pt[0]), int(pt[1])), 7, (255,0,0), -1)
            cv2.line(annotated_img, (int(r_shoulder[0]), int(r_shoulder[1])), (int(r_elbow[0]), int(r_elbow[1])), (255,0,0), 3)
            cv2.line(annotated_img, (int(r_elbow[0]), int(r_elbow[1])), (int(r_wrist[0]), int(r_wrist[1])), (255,0,0), 3)

            cv2.imwrite(save_path, annotated_img)            # 시각화된 이미지 저장
            print(f"[Info] 저장: {save_path}")               # 저장 경로 출력

print('--- 모든 이미지 관절 시각화/저장 완료 ---')            # 전체 완료 메시지 출력


# 코드 전체 요약
# 폴더 구조

# 정상/비정상 두 개의 하위 폴더 내 이미지(jpg, png)들을 대상으로 동작합니다.

# 이미지별로 Mediapipe로 Pose 인식

# 각 이미지를 480x270으로 리사이즈 후 Mediapipe로 사람의 관절(랜드마크)을 인식합니다.

# 왼팔·오른팔 주요 관절 추출 및 시각화

# 왼팔/오른팔(어깨, 팔꿈치, 손목) 좌표만 추출해 원본 이미지 위에 각각 빨강/파랑으로 점+선을 그립니다.

# 결과 이미지 저장

# 시각화 이미지를 별도 폴더에 저장하며, 관절이 인식되지 않은 경우 원본 이미지만 저장합니다.

# 최종적으로 모든 이미지에 대해 관절 시각화 결과를 출력·저장합니다.



[Info] 저장: /content/shoulder_pose_vis/정상/frame_00022.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00023.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00024.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00025.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00026.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00027.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00028.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00029.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00030.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00031.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00032.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00033.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00034.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00035.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00036.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00037.jpg
[Info] 저장: /content/shoulder_pose_vis/정상/frame_00038.jpg
[Info] 저장: /content/shoulder_po

In [ ]:
import cv2                                 # OpenCV (이미지 읽기, 처리)
import mediapipe as mp                     # MediaPipe (관절 인식)
import numpy as np                         # Numpy (수치 연산)
import os                                  # OS (경로, 폴더 관리)
from glob import glob                      # glob (패턴에 맞는 파일 리스트 추출)
import csv                                 # csv (CSV 파일 저장용)

# 1. 데이터 경로 설정
base_dir = '/content/drive/MyDrive/shoulder_unzip'        # 이미지가 들어있는 폴더(상위) 경로
categories = ['정상', '비정상']                           # 두 개의 하위 폴더명 (정상/비정상)
csv_save_path = '/content/shoulder_pose_label.csv'        # 결과 저장할 CSV 경로(이름)

# 2. Mediapipe pose 객체 준비
mp_pose = mp.solutions.pose                              # mediapipe pose 객체

# 3. 결과 저장할 리스트
csv_rows = []                                            # 각 이미지별 결과를 저장할 리스트

# 각 카테고리(정상/비정상)별로 반복
for category in categories:
    input_dir = os.path.join(base_dir, category)                             # 입력 폴더 경로
    img_list = glob(os.path.join(input_dir, '*.jpg')) + \
               glob(os.path.join(input_dir, '*.png'))                        # 해당 폴더 내 모든 이미지 경로 추출

    # mediapipe Pose 객체로 관절 인식 시작
    with mp_pose.Pose(static_image_mode=True, model_complexity=2) as pose:
        for img_path in img_list:                                           # 이미지 파일 하나씩 반복
            img = cv2.imread(img_path)                                      # 이미지 읽기
            if img is None:                                                 # 읽기 실패 시
                print(f'[Error] 이미지 불러오기 실패: {img_path}')
                continue
            img = cv2.resize(img, (480, 270))                               # 480x270 크기로 리사이즈
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)                  # RGB 변환
            result = pose.process(img_rgb)                                  # mediapipe pose 인식 실행
            base_name = os.path.basename(img_path)                          # 파일명만 추출

            if not result.pose_landmarks:                                   # 관절 인식 실패
                print(f'[Warning] Pose 인식 실패: {img_path}')
                # 인식 실패시 CSV에 해당 정보 기록
                csv_rows.append([category, base_name, 'None', 'None', '인식실패'])
                continue

            lm = result.pose_landmarks.landmark                             # 관절 좌표 리스트
            h, w = img.shape[:2]                                            # 이미지 높이, 너비

            # 왼팔 주요 좌표 (어깨, 손목)
            l_shoulder = np.array([lm[11].x * w, lm[11].y * h])
            l_wrist    = np.array([lm[15].x * w, lm[15].y * h])
            # 오른팔 주요 좌표 (어깨, 손목)
            r_shoulder = np.array([lm[12].x * w, lm[12].y * h])
            r_wrist    = np.array([lm[16].x * w, lm[16].y * h])

            # 더 높이 든 팔 찾기 (y좌표가 작을수록 더 높음)
            if l_wrist[1] < r_wrist[1]:                                      # 왼손이 더 위에 있음
                arm = 'left'                                                 # 주요 팔은 왼팔
                # 왼어깨~왼손목 각도 계산 (어깨 기준 오른쪽이 0도, 아래가 +90도)
                angle = np.degrees(np.arctan2(l_wrist[1] - l_shoulder[1],
                                               l_wrist[0] - l_shoulder[0]))
                # 손목이 어깨보다 위에 있으면 정상, 아래면 비정상
                status = "정상" if l_wrist[1] < l_shoulder[1] else "비정상"
            else:                                                            # 오른손이 더 위에 있음
                arm = 'right'                                                # 주요 팔은 오른팔
                angle = np.degrees(np.arctan2(r_wrist[1] - r_shoulder[1],
                                               r_wrist[0] - r_shoulder[0]))
                status = "정상" if r_wrist[1] < r_shoulder[1] else "비정상"

            # 한 줄 결과를 리스트에 추가 (클래스, 파일명, 주요팔, 팔각도, 상태)
            csv_rows.append([category, base_name, arm, angle, status])

# 4. CSV로 저장
with open(csv_save_path, 'w', newline='', encoding='utf-8') as f:           # csv 파일 쓰기 모드로 열기
    writer = csv.writer(f)
    writer.writerow(['class', 'filename', 'main_arm', 'arm_angle', 'status'])  # 헤더 추가
    for row in csv_rows:
        writer.writerow(row)                                                # 결과 한 줄씩 저장

print(f'--- 모든 이미지 자동 라벨/CSV 저장 완료 ---')                          # 완료 출력
print(f'CSV 결과 파일: {csv_save_path}')                                     # 파일 위치 안내
코드 전체 요약
폴더별 이미지 순회:

'정상', '비정상' 폴더 내 모든 jpg, png 이미지를 반복 처리합니다.

# 관절 인식:

# Mediapipe로 어깨/손목 위치를 추출합니다.

# 주요 팔, 각도, 상태 자동 판정:

# 더 높이 든 팔(왼팔/오른팔)을 판별하고, 어깨-손목 각도를 구해,
# 손목이 어깨보다 높으면 "정상", 낮으면 "비정상"으로 판정합니다.

# 결과를 CSV에 저장:

# 클래스(정상/비정상), 파일명, 주요 팔, 각도, 상태(정상/비정상/인식실패) 정보를
# 한 줄씩 기록해 최종적으로 CSV 파일로 저장합니

[Error] 이미지 불러오기 실패: /content/drive/MyDrive/shoulder_unzip/비정상/frame_00072.jpg
--- 모든 이미지 자동 라벨/CSV 저장 완료 ---
CSV 결과 파일: /content/shoulder_pose_label.csv


In [ ]:
# ===========================
# [어깨 Pose 분류 모델 전체 코드] - 정리모드
# ===========================

import pandas as pd                       # 판다스: 데이터프레임 처리
import numpy as np                        # 넘파이: 수치 계산
import os                                 # OS 모듈: 경로 조작
import cv2                                # OpenCV: 이미지 읽기/변환
from sklearn.model_selection import train_test_split  # 데이터 분할
from tensorflow.keras.preprocessing.image import ImageDataGenerator # 이미지 증강
from tensorflow.keras.utils import Sequence           # 커스텀 배치 생성
from tensorflow.keras.applications import MobileNetV2 # 사전학습 모델
from tensorflow.keras import layers, models, optimizers # Keras 모델 구성
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint # 콜백
from sklearn.metrics import classification_report, confusion_matrix   # 성능 평가

# 1. CSV 파일 불러오기
csv_path = '/content/shoulder_pose_label.csv'              # CSV 파일 경로
img_base_dir = '/content/drive/MyDrive/shoulder_unzip'     # 원본 이미지 폴더
df = pd.read_csv(csv_path)                                 # 데이터프레임 로드

# 2. "인식실패" 등 오류/결측치 제거
df = df[df['status'] != '인식실패'].dropna().reset_index(drop=True)   # 인식실패/결측치 제거

# 3. 라벨 0/1로 변환 (정상:1, 비정상:0)
df['label'] = df['status'].map({'정상': 1, '비정상': 0})             # 상태를 0/1 라벨로 변환

# 4. 이미지 파일 경로 생성
df['img_path'] = df.apply(lambda x: os.path.join(img_base_dir, x['class'], x['filename']), axis=1)  # 전체 이미지 경로 생성

# 5. 학습/검증 데이터 분할 (stratify)
train_df, valid_df = train_test_split(
    df, test_size=0.2, random_state=42, stratify=df['label'])      # 라벨 분포 동일하게 분리

# 6. 증강 옵션 준비 (train/valid 따로)
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True) # 학습: 수평반전+정규화
valid_gen = ImageDataGenerator(rescale=1./255)                       # 검증: 정규화만

# 7. 커스텀 데이터셋 클래스 정의
class ShoulderDataset(Sequence):
    def __init__(self, df, batch_size=16, img_size=(224,224), shuffle=True, aug=None):
        self.df = df.reset_index(drop=True)       # 데이터프레임 리셋
        self.batch_size = batch_size              # 배치 크기
        self.img_size = img_size                  # 이미지 크기
        self.shuffle = shuffle                    # 셔플 여부
        self.aug = aug                            # 증강 옵션
        self.on_epoch_end()                       # 최초 섞기(필요시)
    def __len__(self):                            # 전체 배치 수 반환
        return int(np.ceil(len(self.df) / self.batch_size))
    def __getitem__(self, idx):                   # 인덱스별 배치 반환
        batch = self.df.iloc[idx*self.batch_size:(idx+1)*self.batch_size]
        if len(batch) == 0:                       # 빈 배치 예외 처리
            return np.zeros((0, *self.img_size, 3)), np.zeros((0,))
        imgs, labels = [], []
        for _, row in batch.iterrows():
            img = cv2.imread(row['img_path'])     # 이미지 읽기
            if img is None:                       # 실패시, 0이미지 대체
                img = np.zeros((self.img_size[0], self.img_size[1], 3), dtype=np.uint8)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)         # RGB 변환
            img = cv2.resize(img, self.img_size)               # 사이즈 통일
            if self.aug:                                       # 증강 적용
                img = self.aug.random_transform(img)
            imgs.append(img)
            labels.append(row['label'])
        X = np.array(imgs, dtype=np.float32) / 255.            # 0~1 스케일링
        y = np.array(labels, dtype=np.float32)
        return X, y
    def on_epoch_end(self):                   # 에폭 끝마다 셔플
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

# 8. 데이터셋 인스턴스 생성
batch_size = 16
img_size = (224, 224)
train_dataset = ShoulderDataset(train_df, batch_size, img_size, shuffle=True, aug=train_gen)
valid_dataset = ShoulderDataset(valid_df, batch_size, img_size, shuffle=False, aug=valid_gen)

# 9. MobileNetV2 모델 구성 (전이학습)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3), pooling='avg') # feature extractor
base_model.trainable = False                                   # 백본 가중치 고정
model = models.Sequential([
    base_model,                                                # feature extractor
    layers.Dense(64, activation='relu'),                       # FC + ReLU
    layers.Dropout(0.3),                                       # 드롭아웃
    layers.Dense(1, activation='sigmoid')                      # 이진 분류
])
model.compile(optimizer=optimizers.Adam(0.0003),               # Adam 옵티마이저
              loss='binary_crossentropy',                      # 이진 크로스엔트로피
              metrics=['accuracy'])                            # 정확도

# 10. 콜백 설정 (조기종료, 최적 모델 저장)
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)   # val_loss 3번 상승시 종료
mc = ModelCheckpoint('/content/shoulder_best_model.h5', save_best_only=True)    # 최적 가중치 저장

# 11. 모델 학습 (fit)
history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=10,
    callbacks=[es, mc],
    verbose=1
)

# 12. 검증 데이터 최종 정확도 출력
loss, acc = model.evaluate(valid_dataset)
print(f'[최종 검증 정확도] {acc*100:.2f}%')

# 13. 평가 지표(리포트, 혼동행렬) 산출
y_true, y_pred = [], []
for idx in range(len(valid_dataset)):
    X, y = valid_dataset[idx]
    if len(X) == 0:                    # 빈 배치는 건너뜀
        continue
    preds = model.predict(X)
    y_true.extend(y)
    y_pred.extend((preds > 0.5).astype(int).flatten()) # 0.5 기준 이진화

# 리포트 및 혼동행렬 출력
print(classification_report(y_true, y_pred, target_names=['비정상','정상']))
print(confusion_matrix(y_true, y_pred))

# ===========================
# [전체 흐름 요약]
# ---------------------------
# 1. 이미지별 pose 라벨링 결과(CSV)와 원본 이미지를 활용하여 정상/비정상 라벨 분류를 위한 데이터셋을 구성한다.
# 2. 데이터 증강, 커스텀 데이터로더를 적용해 학습/검증용 배치를 만든다.
# 3. MobileNetV2 백본(전이학습) + FC 레이어를 쌓아 이진 분류 모델을 구성한다.
# 4. 조기종료, 베스트 모델 저장 콜백을 적용해 학습한다.
# 5. 최종 검증셋 평가 정확도, 분류 리포트(정밀도/재현율/F1), 혼동행렬을 출력한다.
#    (코드는 마지막 빈 배치 예외처리, 이미지 읽기 오류 방지 등 실전 환경에 맞게 안전하게 작성됨)
# ===========================


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 746ms/step - accuracy: 0.7371 - loss: 0.5402

29/29 ━━━━━━━━━━━━━━━━━━━━ 39s 1s/step - accuracy: 0.7407 - loss: 0.5353 - val_accuracy: 0.9469 - val_loss: 0.2056
Epoch 2/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 725ms/step - accuracy: 0.9767 - loss: 0.1630

29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 960ms/step - accuracy: 0.9764 - loss: 0.1627 - val_accuracy: 0.9558 - val_loss: 0.1447
Epoch 3/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 697ms/step - accuracy: 0.9623 - loss: 0.1287

...

29/29 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.9837 - loss: 0.0810 - val_accuracy: 0.9735 - val_loss: 0.0739
Epoch 5/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 40s 1s/step - accuracy: 0.9751 - loss: 0.0669 - val_accuracy: 0.9735 - val_loss: 0.0828
Epoch 6/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 709ms/step - accuracy: 0.9751 - loss: 0.0649

29/29 ━━━━━━━━━━━━━━━━━━━━ 37s 941ms/step - accuracy: 0.9754 - loss: 0.0644 - val_accuracy: 0.9823 - val_loss: 0.0580
Epoch 7/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 715ms/step - accuracy: 0.9930 - loss: 0.0550

29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 914ms/step - accuracy: 0.9928 - loss: 0.0551 - val_accuracy: 0.9823 - val_loss: 0.0526
Epoch 8/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 740ms/step - accuracy: 0.9759 - loss: 0.0509

29/29 ━━━━━━━━━━━━━━━━━━━━ 27s 923ms/step - accuracy: 0.9759 - loss: 0.0509 - val_accuracy: 0.9823 - val_loss: 0.0506
Epoch 9/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 829ms/step - accuracy: 0.9806 - loss: 0.0534

29/29 ━━━━━━━━━━━━━━━━━━━━ 29s 1s/step - accuracy: 0.9809 - loss: 0.0529 - val_accuracy: 1.0000 - val_loss: 0.0437
Epoch 10/10
29/29 ━━━━━━━━━━━━━━━━━━━━ 28s 953ms/step - accuracy: 0.9820 - loss: 0.0498 - val_accuracy: 0.9823 - val_loss: 0.0462
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 832ms/step - accuracy: 1.0000 - loss: 0.0470
[최종 검증 정확도] 100.00%
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 562ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 578ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 561ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 813ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 957ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
              precision    recall  f1-score   support

         비정상       1.00      1.00      1.00        50
          정상       1.00      1.00      1.00        63

    accuracy                           1.00       113
   macro avg       1.00      1.00      1.00       113
weighted avg       1.00      1.00      1.00       113

[[50  0]
 [ 0 63]]


In [ ]:
# 14. h5 포맷으로 모델 저장
model.save('/content/shoulder_final_model.h5')  # ← 이 경로의 h5파일을 PC로 다운로드!
print('h5 파일로 모델 저장 완료!')


h5 파일로 모델 저장 완료!


In [ ]:
model.save('/content/shoulder_final_model.keras')
print('모델 저장 완료!')

모델 저장 완료!


In [ ]:
from tensorflow.keras.models import load_model
model = load_model('shoulder_best_model.h5')

In [ ]:
# 1. 필요 패키지 import
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# 2. 전처리 함수 정의
def preprocess_for_predict(img_path, img_size=(224, 224)):
    """
    예측용 이미지 전처리 함수
    - img_path: 이미지 경로(str)
    - img_size: (H, W) 튜플, 기본값 (224, 224)
    Return: (1, H, W, 3) 크기의 np.array
    """
    img = cv2.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"이미지 파일을 읽지 못했습니다: {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, img_size)
    img = img.astype(np.float32) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

# 3. 모델 불러오기 (경로는 자신의 파일명에 맞게!)
model = load_model('/content/shoulder_final_model.keras')

# 4. 예측할 이미지 경로 지정
img_path = '/content/drive/MyDrive/shoulder_unzip/비정상/frame_00000.jpg'  # 원하는 이미지 경로로 수정

# 5. 예측
X_pred = preprocess_for_predict(img_path)
pred = model.predict(X_pred)

# 6. 결과 출력
if pred[0,0] > 0.5:
    print("예측 결과: 90도 이상 정상")
else:
    print("예측 결과: 90도 이하 비정상")


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
예측값(확률): 0.07335113734006882
예측 결과: 비정상


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
# y_true와 y_pred는 이미 리스트(또는 np.array) 형태로 만들어졌다고 가정
y_true = np.array(y_true)
y_pred = np.array(y_pred)

# (예측 확률이 아니라, 0/1로 변환된 예측값으로 계산)
mae  = mean_absolute_error(y_true, y_pred)
mse  = mean_squared_error(y_true, y_pred)
rmse = np.sqrt(mse)

mape = np.mean(np.abs((y_true - y_pred) / (y_true + 1e-8))) * 100
r2   = r2_score(y_true, y_pred)

print(f"MAE  : {mae:.4f}")
print(f"MSE  : {mse:.4f}")
print(f"RMSE : {rmse:.4f}")
print(f"MAPE : {mape:.2f}%")
print(f"R²   : {r2:.4f}")


MAE  : 0.0177
MSE  : 0.0177
RMSE : 0.1330
MAPE : 1.77%
R²   : 0.9283


In [ ]:
# ===========================
# [어깨 각도별 평가/추천/점수 계산 함수] - 정리모드
# ===========================

import os

# 각도별 제한 구조/강화 근육/점수 테이블 정의
ANGLE_EVAL_TABLE = [
    # (start, end, 제한 구조, 강화 근육, 점수 구간(%) )
    (90, 100,   "극상근, 견갑하근, 상완이두근(긴장), 광배근, 소원형근",     "전거근, 하부승모근, 하부삼각근",            (10, 30)),
    (100, 110,  "견갑하근, 상부승모근, 상완이두근, 소원형근, 광배근",         "전거근, 하부승모근, (중부·후부 삼각근)",     (30, 40)),
    (110, 120,  "상부승모근, 광배근, 소흉근",                              "전거근, 하부승모근",                        (30, 40)),
    (120, 130,  "견갑하근,상부승모근,광배근,소흉근",                      "전거근, 하부·중부승모근",                   (40, 50)),
    (130, 140,  "견갑하근, 광배근, 삼각근,소흉근",                        "전거근, 하부·중부승모근",                   (40, 50)),
    (140, 150,  "상부승모근, 광배근, 소흉근, 사각근",                     "전거근, 하부승모근",                        (60, 70)),
    (150, 160,  "광배근, 소흉근, 사각근,견갑하근",                        "전거근, 하부승모근, 상부승모근,회전근개(외회전)", (70, 80)),
    (160, 170,  "광배근, 소흉근,사각근, 견갑하근",                        "전거근, 하부승모근,회전근개(외회전)",        (80, 90)),
    (170, 180,  "소흉근, 광배근, 견갑하근",                              "전거근, 하부승모근,회전근개(외회전)",        (90, 100)),
    (180, 180,  "정상",                                               "정상",                                  (100, 100)),
]

def shoulder_eval_postprocess(angle: float):
    """
    입력된 어깨 각도(angle, degree)에 따라 제한 구조/강화근육/점수를 산출하는 함수

    Parameters:
        angle (float): 어깨 가동범위 각도 (degree, 도 단위)

    Returns:
        dict: {
            'angle': 입력 각도,
            'limitation': 제한 구조(근육),
            'recommendation': 강화 추천 근육,
            'score_percent': 점수(%)
        }
    """
    result = {
        "angle": angle,              # 입력 각도 그대로 기록
        "limitation": "측정 불가",   # 기본값(매칭 실패 시)
        "recommendation": "측정 불가",
        "score_percent": 0
    }
    # 각도 구간별로 테이블 조회
    for start, end, limitation, recommend, percent_range in ANGLE_EVAL_TABLE:
        # angle이 구간에 포함되거나 180 이상이면 해당 구간 선택
        if (start <= angle < end) or (angle >= 180):
            result["limitation"] = limitation                # 제한 구조(근육) 기록
            result["recommendation"] = recommend             # 강화 근육 추천 기록
            if end > start:
                # 점수 구간 내에서 비율에 따라 점수 선형 보간
                percent = percent_range[0] + (percent_range[1] - percent_range[0]) * ((angle - start) / (end - start))
            else:
                # 시작과 끝이 같은 경우, 구간 점수 그대로
                percent = percent_range[0]
            result["score_percent"] = round(percent, 1)      # 소수점 1자리로 반올림
            break
    return result

# ===========================
# [전체 흐름 요약]
# ---------------------------
# 1. ANGLE_EVAL_TABLE에서 어깨 각도별 제한되는 구조(근육), 강화 추천 근육, 점수 구간(%) 정보를 정의한다.
# 2. 입력 각도가 테이블의 구간 내에 속하면, 그 구간의 제한 구조/추천근육/점수(구간 내 보간)를 반환한다.
# 3. 180도 이상은 정상, 구간에 포함 안 되면 "측정 불가"로 리턴한다.
# 4. 결과는 angle, limitation, recommendation, score_percent 키를 가진 dict로 반환한다.
# ===========================
